In [ ]:
!pip install torchattacks
!pip install pandas
!pip install openpyxl
!pip install numpy
!pip install plot 
!pip install numpy

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchattacks
from torch.optim import lr_scheduler
import time

In [ ]:
import Sources.DefaultModel as Util
import Sources.GTSRBModel as GTSRBModel
import Sources.TestUtils as TestUtils

In [ ]:
Device = Util.HardwareIdentification()
DimImage = 50
TrainLoader,ValidationLoader, TestLoader = GTSRBModel.ImportData(1, DimImage)
print(len(TrainLoader.dataset), len(ValidationLoader.dataset), len(TestLoader.dataset))

In [ ]:
EPOCHS = 5
LEARNING_RATE = 0.0008
INPUT_DIM = 3*DimImage*DimImage
OUTPUT_DIM = 43
Model = GTSRBModel.GTSRB_MODEL(INPUT_DIM,OUTPUT_DIM, Device).to(Device)

optimizer = torch.optim.Adam(params=Model.parameters(),lr=LEARNING_RATE)
lr_s = lr_scheduler.LinearLR(optimizer,start_factor=1.0,end_factor=0.5,total_iters=10)
loss = nn.CrossEntropyLoss()

In [ ]:
%%script false --no-raise-error
Model.compile(train_data=TrainLoader,validation_data=ValidationLoader,epochs=EPOCHS,loss_function=loss,optimizer=optimizer,learning_rate_scheduler=lr_s)
ModelPath = 'Models/ModelSignal.torch'
torch.save(Model.state_dict(), ModelPath)

In [ ]:
Model.load_state_dict(torch.load('Models/ModelSignal.torch'))

In [ ]:
Util.TestLoop(TestLoader, Model, Device)
X, y = next(iter(TestLoader))

In [ ]:
Coef=5
Activation=4
Prob = 2
Similarity = 0.2
Init=0
lr = 0.2
Steps = 1000

data = []
start_time = time.time()
data = TestUtils.TestLoopC(Model, Steps, lr, Coef, Activation, Prob, Similarity, Init, X, y, Device, 1000, 0.1, 1.5)
end_time = time.time()

TestUtils.SaveTime(start_time, end_time, "Results/GTSRB/ComputationTimeResults.txt")
TestUtils.SaveData(data, "Results/GTSRB/Results.xlsx")

In [ ]:
TestUtils.SaveImages("Results/GTSRB/InitialImages/", X, y)

In [ ]:
Coef=5
Activation=4
Prob = 2
Similarity = 0.2
Init=0
lr = 0.2
Steps = 10000

TestUtils.SaveLoopC(Model, Steps, lr, Coef, Activation, Prob, Similarity, Init, X, y, Device, 1000, 0.1, 1.5, "Results/GTSRB/Images/")

In [ ]:
Coef=5
Activation=4
Prob = 2
Similarity = 0.2
Init=0
lr = 0.2

TestedIterations = [100, 1000]

for i in range((len(TestedIterations))):
    data = []
    start_time = time.time()
    data = TestUtils.TestLoopC(Model, TestedIterations[i], lr, Coef, Activation, Prob, Similarity, Init, X, y, Device, 1000, 0.1, 1.5)
    end_time = time.time()

    TestUtils.SaveTime(start_time, end_time, "Results/GTSRB/TimeXS" + str(TestedIterations[i]) +".txt")
    TestUtils.SaveData(data, "Results/GTSRB/BestXS" + str(TestedIterations[i]) + ".xlsx")
    

In [ ]:
dataJSMA = []

start_time = time.time()
Data = TestUtils.TestJSMA(1,1, Model, X, y, Device)
end_time = time.time()

TestUtils.SaveTime(start_time, end_time, "Results/GTSRB/TimeJSMAS.txt")
TestUtils.SaveData(Data, "Results/GTSRB/BestJSMAS.xlsx")

In [ ]:
TestUtils.SaveJSMA(1,1, Model, X, y, Device, "Results/GTSRB/JSMAImages/")